In [3]:
from pymongo import MongoClient
import pandas as pd
from bson.objectid import ObjectId
import numpy as np
from collections import defaultdict
from tqdm import tnrange, tqdm_notebook

Однако также не доделан последний абзац про Крым. Там нельзя так говорить. Тема очень грязная; в основном, там тексты про зимнее время, на которые и есть большинство комментариев, а также примешивается Pussy Riot. Нельзя писать то, чего нет. Продление дедлайна уже объявлено, я могу закончить 13го в первой половине дня, если будет следующая информация.

Я предлагаю Вам разделить топ тексты Крыма (из файла) на три группы: 1. Крым. 2. Pussy Riot 3. Остальное (зимнее время и какая-то фигня). Про каждую группу нужны такие поля:
1. Количество новостей
2. Количество комментариев.
3. Индекс негативности-позитивности по этой группе комментариев (соотв., без домножения на вероятность темы).
4. Индекс controversy (тоже без домножения на тему). 

Посмотрим, что получится. Моя гипотеза: тексты Крыма вообще будут мало комментированы, причем не особо позитивно. А вот что там с другими подтемами, нет гипотез, я их не читала.

In [6]:
db = MongoClient().thesis

In [4]:
urls = ["http://www.omskinform.ru/news/66194", "http://www.omskinform.ru/news/64561", "http://www.omskinform.ru/news/65062", "http://www.bk55.ru/news/article/31930/", "http://www.omskinform.ru/news/64513", "http://www.omskinform.ru/news/64517", "http://www.omskinform.ru/news/64514", "http://www.bk55.ru/news/article/26127/", "http://gorod55.ru/news/article/0e86939b-dee5-46d0-fdad-56bbf989e7d6", "http://ngs55.ru/news/1395788/view/", "http://www.bk55.ru/news/article/30523/", "http://gorod55.ru/news/article/c5233ef4-2149-4778-a95e-573ac98ae311", "http://www.omskinform.ru/news/64516", "http://www.omskinform.ru/news/62121", "http://www.omskinform.ru/news/62127", "http://www.bk55.ru/news/article/31782/", "http://www.bk55.ru/news/article/36030/", "http://www.bk55.ru/news/article/29863/", "http://ngs55.ru/news/1791151/view/", "http://www.omskinform.ru/news/59030", "http://www.bk55.ru/news/article/31257/", "http://www.omskinform.ru/news/64510", "http://www.bk55.ru/news/article/29914/", "http://www.bk55.ru/news/article/30506/", "http://gorod55.ru/news/article/98aada45-1a29-4106-a9a3-790d8a78f043", "http://www.bk55.ru/mc2/news/article/1628", "http://www.bk55.ru/news/article/29465/", "http://www.bk55.ru/news/article/30486/", "http://gorod55.ru/news/article/8341d7b6-7632-4bdf-ef29-8e9ab99e4554", "http://gorod55.ru/news/article/b5050e39-49bd-41bb-f9c6-fb5c79dda28f", "http://www.bk55.ru/mc2/news/article/2343", "http://www.bk55.ru/news/article/29957/", "http://www.bk55.ru/news/article/34988/", "http://www.bk55.ru/news/article/31828/", "http://www.bk55.ru/news/article/21649/", "http://www.omskinform.ru/news/67556", "http://www.bk55.ru/news/article/29974/", "http://www.bk55.ru/news/article/26179/", "http://www.omskinform.ru/news/64506", "http://www.bk55.ru/news/article/26828/", "http://ngs55.ru/news/1851441/view/", "http://gorod55.ru/news/article/d46a7b75-28ee-42eb-d5d7-769bbc24c467", "http://www.omskinform.ru/news/65088", "http://www.bk55.ru/news/article/21683/", "http://www.omskinform.ru/news/64541", "http://www.bk55.ru/news/article/21525/", "http://www.omskinform.ru/news/64625", "http://www.bk55.ru/news/article/34050/", "http://gorod55.ru/news/article/6076a586-2880-41ec-a53e-5552b512a463", "http://www.bk55.ru/news/article/36971/"]

In [16]:
# %load topics_by_id.py
def get_topic_by_id(id):
    TOPICS_NUMBERING = {
        0: "Accidents with children",
        1: "Local authorities: appointments, resignations & statements",
        2: "*Urban development (misc)",
        3: "IT & military high tech",
        4: "Fires",
        5: "Urban demography & housing payments",
        6: "Police actions drug, alcohol & counterfeit money crimes",
        7: "Omsk region industrial developemnt",
        8: "Ads of banking services",
        9: "Holidays and VIP-weddingss",
        10: "Trials on economic crime",
        11: "Public transport and traffic",
        12: "Weather",
        13: "Yury Gamburg resignation",
        14: "*NATO warships in Black sea, Russian rocket launch & contests",
        15: "Regional parliament activities",
        16: "Urban landscaping & greening",
        17: "Schools, orphanages & child charity",
        18: "Car sales",
        19: "Car accidents",
        20: "Education",
        21: "Real estate: contruction",
        22: "Art and literature",
        23: "Street & bridge reconstruction & maintenance",
        24: "Housing & the case of disabled Akhmetov",
        25: "Stray dogs & doghunters",
        26: "Urban events & openings",
        27: "Crimea accession",
        28: "Concerts",
        29: "Sport, hockey",
        30: "Hockey",
        31: "Control & regulation of enterprises",
        32: "Regional taxes & fuel prices",
        33: "Missing persons announcements",
        34: "Theaters & festivals",
        35: "*Omsk media, plants & animals",
        36: "Local authorities: appointments, resignations & statements",
        37: "Macroeconomic events: currency rates & oil prices",
        38: "Housing: heating",
        39: "Russia, Ukraine & US international relations",
        40: "Beauty contests & their winners",
        41: "Arbitration court and the Mostovik case",
        42: "*Military holidays",
        43: "Criminal news",
        44: "Movies and Movie stars",
        45: "Olympic Games 2014 & Omsk athletes",
        46: "Uninterpretable",
        47: "Ivan Klimov’s murder",
        48: "Abridgments of traffic law",
        49: "Regional elections and misc."
    }
    return TOPICS_NUMBERING[id]

In [19]:
results = defaultdict(int)

for num, url in enumerate(urls):
    doc = db.final_db.find_one({"url": url})
    
    sent_scores = [comment[-1] for comment in doc["comments"]]
    comments_num = len(doc["comments"])
    
    if comments_num:
        avg_comment_sent = sum(sent_scores)/comments_num
        avg_comment_std = np.std(sent_scores)

        results["comments_num"] += prob * comments_num           
        results["avg_comment_sent"] += prob * avg_comment_sent
        results["avg_comment_std"] += prob * avg_comment_std

In [20]:
results

defaultdict(int,
            {'comments_num': 0.09314814814814822,
             'avg_comment_sent': 0.0015300015297133433,
             'avg_comment_std': 0.0035818401730726038})

In [22]:
crimea_inside = pd.read_excel("inside_crimea_topic.xlsx")

In [30]:
crimea_inside.fillna(0, inplace=True)

In [41]:
db.final_db.find_one({"url": urls[0]})

{'_id': ObjectId('5489ddf737a808115d1eda4e'),
 'title': 'Пиво с изображением героев войны решили не выпускать',
 'url': 'http://www.omskinform.ru/news/66194',
 'topics': {'0': 0.00013605442176870726,
  '1': 0.00013605442176870726,
  '2': 0.00013605442176870726,
  '3': 0.00013605442176870726,
  '4': 0.00013605442176870726,
  '5': 0.00013605442176870726,
  '6': 0.00013605442176870726,
  '7': 0.00013605442176870726,
  '8': 0.00013605442176870726,
  '9': 0.00013605442176870726,
  '10': 0.02742842740259543,
  '11': 0.00013605442176870726,
  '12': 0.00013605442176870726,
  '13': 0.00013605442176870726,
  '14': 0.00013605442176870726,
  '15': 0.00013605442176870726,
  '16': 0.00013605442176870726,
  '17': 0.00013605442176870726,
  '18': 0.00013605442176870726,
  '19': 0.00013605442176870726,
  '20': 0.00013605442176870726,
  '21': 0.00013605442176870726,
  '22': 0.00013605442176870726,
  '23': 0.00013605442176870726,
  '24': 0.00013605442176870726,
  '25': 0.00013605442176870726,
  '26': 0.00

In [90]:
def f(row):
#     print(row)
    if row["Про Крым"] == 1:
        doc = db.final_db.find_one({"url": row["URL"]})
        return len(doc["comments"])

In [91]:
crimea_inside\
    .apply(f, axis=1).sum()

82.0

In [93]:
for i in ["Про Крым", "Про Pussy Riot", "Часовые пояса", "Пиво, ВОВ", "Оппозиция, Навальный, 25/17"]:
    sent_scores = []
    for url in crimea_inside[crimea_inside[i] == 1]["URL"].tolist():
        doc = db.final_db.find_one({"url": url})
        sent_scores += [comment[-1] for comment in doc["comments"]]
#         sent_scores_sum = np.sum(sent_scores)

    std = np.std(sent_scores)
    mean = np.mean(sent_scores)
    print(i, std, mean)

Про Крым 0.5192602160010675 0.32926829268292684
Про Pussy Riot 0.6745330214944028 0.11290322580645161
Часовые пояса 0.6130613945004937 0.31063829787234043
Пиво, ВОВ 0.7582417527118612 -0.008333333333333333
Оппозиция, Навальный, 25/17 0.6959705453537527 0.375


In [27]:
for url in crimea_inside[crimea_inside["Про Крым"] == 1]["URL"].tolist():
    

['http://www.omskinform.ru/news/64561',
 'http://www.omskinform.ru/news/65062',
 'http://www.omskinform.ru/news/64513',
 'http://www.omskinform.ru/news/64517',
 'http://www.omskinform.ru/news/64514',
 'http://gorod55.ru/news/article/0e86939b-dee5-46d0-fdad-56bbf989e7d6',
 'http://gorod55.ru/news/article/c5233ef4-2149-4778-a95e-573ac98ae311',
 'http://www.omskinform.ru/news/64516',
 'http://www.bk55.ru/news/article/29863/',
 'http://www.omskinform.ru/news/64510',
 'http://www.bk55.ru/news/article/29914/',
 'http://www.bk55.ru/news/article/30506/',
 'http://www.bk55.ru/news/article/29465/',
 'http://www.bk55.ru/news/article/30486/',
 'http://gorod55.ru/news/article/b5050e39-49bd-41bb-f9c6-fb5c79dda28f',
 'http://www.bk55.ru/mc2/news/article/2343',
 'http://www.bk55.ru/news/article/29957/',
 'http://www.bk55.ru/news/article/29974/',
 'http://www.omskinform.ru/news/64506',
 'http://www.omskinform.ru/news/64541',
 'http://www.omskinform.ru/news/64625']